In [2]:
import json

file = open("input/cards.json")
cards = json.load(file)
file.close()

In [12]:
def typeToC(type):
    if type == "Challenger":
        return "CHALLENGER"
    if type == "Grotto":
        return "GROTTO"
    if type == "Wish":
        return "WISH"
    return "BEAST"

def slashToZero(v):
    return 0 if v == "-" else v

def cardToC(card):
    return f"""{{
            {typeToC(card["Type"])},
            {card["Type"][:-6].upper() if typeToC(card["Type"]) == "BEAST" else "NONE"},
            {"RARE" if card["Rarity"] == "R" else "COMMON"},
            {"true" if card["Epic"] == "TRUE" else "false"},
            {slashToZero(card["Cost"])},
            {slashToZero(card["Power"])},
            {slashToZero(card["Goal"])},
            "{card["Name"][:12]}",
            "{card["Name"]}",
            "{json.dumps(card["Effect"])[1:-1]}",
            "{json.dumps(card["Fun"])[1:-1]}",
        }},
        """

def cardsToC(cards):
    ret = ""
    for card in cards:
        ret += cardToC(card)
    return ret

card_beast_types = set()

for card in cards:
    if typeToC(card["Type"]) == "BEAST":
        card_beast_types.add(card["Type"][:-6])

def cardBeastTypesToC():
    ret = ""
    for i,t in enumerate(card_beast_types):
        ret += f"""{t.upper()} = {i},
        """
    return ret

def cardBeastTypesToCStrings():
    ret = ""
    for i,t in enumerate(card_beast_types):
        ret += f""""{t}",
        """
    return ret

In [13]:


file = open("output/gb_cards.h", "w")
file.write("""
#ifndef GB_CARDS_H
#define GB_CARDS_H

namespace gb {
    enum card_type {
        CHALLENGER = 0,
        GROTTO,
        WISH,
        BEAST,
        NUM_CARD_TYPES
    };
    
    constexpr char card_type_strings[NUM_CARD_TYPES][12] = {
        "Challeneger",
        "Grotto",
        "Wish",
        "Beast",
    };
           
    enum card_beast_type {
        """ + cardBeastTypesToC() + """NONE,
        NUM_BEAST_TYPES
    };

    constexpr char card_beast_type_strings[NUM_BEAST_TYPES][13] {
        """ + cardBeastTypesToCStrings() + """"",
    };

    enum card_rarity {
        COMMON = 0,
        RARE,
    };
    
    struct card {
        card_type type;
        card_beast_type beast_type;
        card_rarity rarity;
        bool epic;
        int cost;
        int power;
        int goal;
        char short_name[13];
        char long_name[25];
        char effect[424];
        char flavor[157];
    };

    constexpr card cards[200] = {
        """+ cardsToC(cards) + """
    };
}

#endif
""")

file.close()